In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

Текстовый файл `sequences.fasta` содержит набор последовательностей белков гистонов. Идентификатор последовательности содержит информацию о виде живого организма, идентификатор записи в базе данных NCBI, тип гистона, а также вариант гистона. Например, идентификатор `Drosophila|NP_724343.1|H2A|cH2A` означает, что последовательность из плодовой мухи дрозофилы, идентификатор записи в базе данных `NCBI NP_724343.1`, тип гистона H2A, вариант гистона cH2A. Создайте словарь (vocab) уникальных символов, входящих в состав последовательностей (символ “-” должен быть в составе словаря), и объедините его с уникальным набором типов гистонов.

Сколько элементов получилось в переменной vocab?

In [14]:
vocab_seq = set()
histone_types = set()

with open("sequences.fasta", "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith(">"):
            parts = line[1:].split("|")
            if len(parts) >= 3:
                histone_types.add(parts[2])
        else:
            vocab_seq.update(line)  
            
vocab = vocab_seq.union(histone_types)

print("Количество элементов в vocab:", len(vocab))

Количество элементов в vocab: 28


Создайте два словаря: в первом словаре ключом является символ, а значением его порядковый номер в словаре vocab; во втором словаре - наоборот. Первый словарь понадобится для того, чтобы закодировать последовательность и целевые значения (тип гистона) в виде тензора. Второй словарь, чтобы “превратить” тензор (например, полученный в результате предсказания) с исходный символ или тип гистона. Используя первый словарь, получите список тензоров, в котором каждый тензор - это закодированная последовательность, и тензор, хранящий закодированные целевые значения (типы гистонов для каждой последовательности). Сколько последовательностей получилось в предобработанном датасете (длина списка тензоров)?

In [ ]:
vocab_seq = set()
histone_types = set()

sequences = []
labels = []

with open("sequences.fasta", "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith(">"):
            parts = line[1:].split("|")
            if len(parts) >= 4:
                current_histone = parts[2]
                histone_types.add(current_histone)
        else:
            sequences.append(line)
            labels.append(current_histone)
            vocab_seq.update(line)

char_to_idx = {ch: idx for idx, ch in enumerate(sorted(vocab_seq))}
idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}
histone_to_idx = {h: idx for idx, h in enumerate(sorted(histone_types))}
idx_to_histone = {idx: h for h, idx in histone_to_idx.items()}
encoded_sequences = [torch.tensor([char_to_idx[ch] for ch in seq], dtype=torch.long) for seq in sequences]
encoded_labels = torch.tensor([histone_to_idx[label] for label in labels], dtype=torch.long)

print("Количество последовательностей в датасете:", len(encoded_sequences))

Количество последовательностей в датасете: 9040


В этом задании вам необходимо написать рекуррентную нейросеть с механизмом внимания с использованием PyTorch. В качестве входного слоя создайте Embedding. Далее создайте 1 рекуррентный слой. В качестве слоя внимания используйте линейную трансформацию. В качестве выходного слоя используйте линейную трансформацию.

In [ ]:
embedding_dim = 5
hidden_dim = 9

class RNNWithAttentionModel(nn.Module):
    def __init__(self, vocab_size, random_seed=5):
        super(RNNWithAttentionModel, self).__init__()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)  
        out, _ = self.rnn(x)  
        attention_scores = self.attention(out).squeeze(2)  
        attention_weights = F.softmax(attention_scores, dim=1) 
        context = torch.sum(out * attention_weights.unsqueeze(2), dim=1)  
        out = self.fc(context)  
        return out

Запустите обучение нейросети на 700 эпохах. Используйте значение кросс-энтропии в качестве функции потерь и адаптивный алгоритм Adam со скоростью обучения 0.01 в качестве алгоритма оптимизации.

In [8]:
embedding_dim = 5
hidden_dim = 9
epochs = 700
learning_rate = 0.01
batch_size = 16

class HistoneDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_seqs = nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.long)
    return padded_seqs, labels

dataset = HistoneDataset(encoded_sequences, encoded_labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
model = RNNWithAttentionModel(vocab_size=len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()
        output = model(x_batch)         
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 50 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 1/700, Loss: 803.4678
Epoch 50/700, Loss: 633.6779
Epoch 100/700, Loss: 591.9705
Epoch 150/700, Loss: 606.2118
Epoch 200/700, Loss: 599.1180
Epoch 250/700, Loss: 595.0021
Epoch 300/700, Loss: 598.0742
Epoch 350/700, Loss: 593.9923
Epoch 400/700, Loss: 592.8188
Epoch 450/700, Loss: 592.2310
Epoch 500/700, Loss: 620.3305
Epoch 550/700, Loss: 609.3977
Epoch 600/700, Loss: 593.7352
Epoch 650/700, Loss: 587.5502
Epoch 700/700, Loss: 588.9373


In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x_batch, y_batch in dataloader:
        output = model(x_batch)
        _, predicted = torch.max(output, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total * 100
print(f"Final Accuracy: {accuracy:.2f}%")

Final Accuracy: 57.27%


Используя обученную модель нейросети предскажите тип гистона для последовательности: 

seq_fasta = '''>Pan|XP_003311177.1|HTYPE|HVARIANT 

MSGRGKQGGKARTKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLT 

AEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGKVTIAQGGVLPNIQAVLLPKK 

TESHHKAKGK'''

In [ ]:
seq_fasta = '''>Pan|XP_003311177.1|HTYPE|HVARIANT 

MSGRGKQGGKARTKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGAGAPVYLAAVLEYLT 

AEILELAGNAARDNKKTRIIPRHLQLAIRNDEELNKLLGKVTIAQGGVLPNIQAVLLPKK 

TESHHKAKGK'''

sequence = ''.join([line.strip() for line in seq_fasta.splitlines() if not line.startswith(">")])
encoded_seq = [histone_to_idx[c] for c in sequence]
input_tensor = torch.tensor(encoded_seq, dtype=torch.long).unsqueeze(0)  
model.eval()

with torch.no_grad():
    output = model(input_tensor)
    predicted_idx = torch.argmax(output, dim=1).item()
    predicted_histone = idx_to_histone[predicted_idx]

print(f"Предсказанный тип гистона: {predicted_histone}")

Предсказанный тип гистона: H2A
